<a href="https://colab.research.google.com/github/sanatsingh/CNN/blob/master/StyleTransfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Style Transfer in PyTorch

In this notebook you can see how style from a certain image can be extracted and can be applied to the content of another image.

A 19-layer VGG Network, which is comprised of a series of convolutional and pooling layers, and a few fully-connected layers is used for feature and content extraction. The convolutional layers are named by stack and their order in the stack. Conv_1_1 is the first convolutional layer that an image is passed through, in the first stack. Conv_2_1 is the first convolutional layer in the second stack. The deepest convolutional layer in the network is conv_5_4.

In [0]:
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import torch
import torch.optim as optim
from torchvision import transforms, models